# 2️⃣ Train: 모델 학습 파이프라인

이 노트북은 다음을 수행합니다:
1. 모델 프루닝 (Pruning)
2. 지식 증류 파인튜닝 (Output KD, Feature KD, FaKD) - **200 에폭 × 5회 = 총 1000 에폭**
3. 모델 재구성 (Rebuild)
4. 학습 결과 시각화

**학습 방식**: 각 KD 방법은 200 에폭씩 5번 반복 실행하여 총 1000 에폭을 학습합니다.
- 로그 파일이 자동으로 이어서 기록됩니다
- 각 파트 완료 후 그래프가 표시됩니다
- 1000 에폭 완료 시 비교 그래프가 자동 생성됩니다

## 2.0 환경 변수 설정

In [ ]:
import os
import sys

# 상대 경로 설정 (노트북이 ai/ 폴더에 있다고 가정)
project_dir = os.path.dirname(os.path.abspath('__file__'))  # ai/
if not project_dir or project_dir == '.':
    project_dir = os.getcwd()

# 가상환경 경로 설정
home_dir = os.path.expanduser('~')
venv_path = os.path.join(home_dir, 'my_ssl/ssl_env')
site_packages = os.path.join(venv_path, 'lib', 'python3.12', 'site-packages')
python_exec = os.path.join(venv_path, 'bin', 'python')

# sys.path 설정
if site_packages not in sys.path:
    sys.path.insert(0, site_packages)

# 프로젝트 디렉토리로 이동
%cd {project_dir}

print(f"✅ 환경 설정 완료")
print(f"   Python: {python_exec}")
print(f"   Project: {project_dir}")

# 필수 라이브러리 체크
print("\n📦 라이브러리 체크:")
required_libs = ['pandas', 'matplotlib', 'numpy', 'torch', 'tqdm', 'yaml', 'PIL']
missing_libs = []

for lib in required_libs:
    try:
        if lib == 'yaml':
            __import__('yaml')
        elif lib == 'PIL':
            __import__('PIL')
        else:
            __import__(lib)
        print(f"   ✅ {lib}")
    except ImportError:
        print(f"   ❌ {lib} - 설치 필요!")
        missing_libs.append(lib)

if missing_libs:
    print(f"\n⚠️ 누락된 라이브러리: {', '.join(missing_libs)}")
    print(f"   설치: pip install {' '.join(missing_libs)}")
else:
    print("\n✅ 모든 라이브러리 준비 완료!")

## 2.1 이전 결과 정리 (선택)

In [ ]:
# ⚠️ 주의: 이전 학습 결과를 삭제합니다!
# 필요한 경우에만 주석 해제하여 실행하세요.

# print("🗑️ 이전 파일 정리 중...")
# !rm -f weights/catanet_pruned.pth weights/catanet_pruning_masks.pth
# !rm -f weights/finetuned_*.pth weights/rebuilt_*.pth
# !rm -rf results/logs/* results/plots/*/* results/summary/*
# print("✅ 정리 완료")

print("💡 이전 결과를 삭제하려면 위 주석을 해제하세요.")
print("📁 결과 저장 위치: results/")

## 2.2 모델 프루닝 (Pruning)

In [ ]:
print("🚀 [STEP 1] 모델 프루닝 시작...")
print("="*60)

!{python_exec} run_catanet_pruning.py --config config_catanet.yml

print("\n✅ 프루닝 완료")
print("📁 생성된 파일:")
!ls -la weights/catanet_pruned.pth weights/catanet_pruning_masks.pth 2>/dev/null || echo "파일 없음"

---
## 2.3 지식 증류 - Output KD (200 에폭 × 5회 = 1000 에폭)

In [ ]:
# Output KD - Round 1/5 (에폭 1-200)
print("🚀 Output KD [1/5] 시작... (에폭 1-200)")
print("="*60)

!{python_exec} finetune_pruned_model.py \
    --config config_catanet.yml \
    --teacher_weights weights/CATANet-L_x2.pth \
    --pruned_weights weights/catanet_pruned.pth \
    --save_path weights/finetuned_output.pth \
    --distillation_type output \
    --epochs 200 \
    --lr 1e-4 \
    --alpha 0.8

print("\n✅ Output KD [1/5] 완료!")

In [ ]:
# Output KD - Round 2/5 (에폭 201-400)
print("🚀 Output KD [2/5] 시작... (에폭 201-400)")
print("="*60)

!{python_exec} finetune_pruned_model.py \
    --config config_catanet.yml \
    --teacher_weights weights/CATANet-L_x2.pth \
    --pruned_weights weights/catanet_pruned.pth \
    --save_path weights/finetuned_output.pth \
    --distillation_type output \
    --epochs 200 \
    --lr 1e-4 \
    --alpha 0.8

print("\n✅ Output KD [2/5] 완료!")

In [ ]:
# Output KD - Round 3/5 (에폭 401-600)
print("🚀 Output KD [3/5] 시작... (에폭 401-600)")
print("="*60)

!{python_exec} finetune_pruned_model.py \
    --config config_catanet.yml \
    --teacher_weights weights/CATANet-L_x2.pth \
    --pruned_weights weights/catanet_pruned.pth \
    --save_path weights/finetuned_output.pth \
    --distillation_type output \
    --epochs 200 \
    --lr 1e-4 \
    --alpha 0.8

print("\n✅ Output KD [3/5] 완료!")

In [ ]:
# Output KD - Round 4/5 (에폭 601-800)
print("🚀 Output KD [4/5] 시작... (에폭 601-800)")
print("="*60)

!{python_exec} finetune_pruned_model.py \
    --config config_catanet.yml \
    --teacher_weights weights/CATANet-L_x2.pth \
    --pruned_weights weights/catanet_pruned.pth \
    --save_path weights/finetuned_output.pth \
    --distillation_type output \
    --epochs 200 \
    --lr 1e-4 \
    --alpha 0.8

print("\n✅ Output KD [4/5] 완료!")

In [ ]:
# Output KD - Round 5/5 (에폭 801-1000)
print("🚀 Output KD [5/5] 시작... (에폭 801-1000)")
print("="*60)

!{python_exec} finetune_pruned_model.py \
    --config config_catanet.yml \
    --teacher_weights weights/CATANet-L_x2.pth \
    --pruned_weights weights/catanet_pruned.pth \
    --save_path weights/finetuned_output.pth \
    --distillation_type output \
    --epochs 200 \
    --lr 1e-4 \
    --alpha 0.8

print("\n✅ Output KD [5/5] 완료! (총 1000 에폭)")

In [ ]:
# Output KD 그래프 표시
import pandas as pd
import matplotlib.pyplot as plt
import os

print("📊 Output KD 학습 결과")
print("="*60)

log_path = 'results/logs/output_kd.csv'
if os.path.exists(log_path):
    df = pd.read_csv(log_path)
    print(f"현재 에폭: {len(df)}/1000")
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Loss 그래프
    axes[0].plot(df['epoch'], df['total_loss'], label='Total Loss', color='blue')
    axes[0].plot(df['epoch'], df['task_loss'], label='Task Loss', color='green', alpha=0.7)
    axes[0].plot(df['epoch'], df['distill_loss'], label='Distill Loss', color='red', alpha=0.7)
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Output KD - Training Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # PSNR 그래프
    axes[1].plot(df['epoch'], df['val_psnr'], label='Val PSNR', color='purple', linewidth=2)
    axes[1].axhline(y=df['val_psnr'].max(), color='red', linestyle='--', alpha=0.5, 
                    label=f'Best: {df["val_psnr"].max():.2f}')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('PSNR (dB)')
    axes[1].set_title('Output KD - Validation PSNR')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📈 Best PSNR: {df['val_psnr'].max():.4f} (에폭 {df.loc[df['val_psnr'].idxmax(), 'epoch']})")
else:
    print("❌ Output KD 로그 파일이 없습니다.")

---
## 2.4 지식 증류 - Feature KD (200 에폭 × 5회 = 1000 에폭)

In [ ]:
# Feature KD - Round 1/5 (에폭 1-200)
print("🚀 Feature KD [1/5] 시작... (에폭 1-200)")
print("="*60)

!{python_exec} finetune_pruned_model.py \
    --config config_catanet.yml \
    --teacher_weights weights/CATANet-L_x2.pth \
    --pruned_weights weights/catanet_pruned.pth \
    --save_path weights/finetuned_feature.pth \
    --distillation_type feature \
    --epochs 200 \
    --lr 1e-4 \
    --alpha 0.5 \
    --beta 1

print("\n✅ Feature KD [1/5] 완료!")

In [ ]:
# Feature KD - Round 2/5 (에폭 201-400)
print("🚀 Feature KD [2/5] 시작... (에폭 201-400)")
print("="*60)

!{python_exec} finetune_pruned_model.py \
    --config config_catanet.yml \
    --teacher_weights weights/CATANet-L_x2.pth \
    --pruned_weights weights/catanet_pruned.pth \
    --save_path weights/finetuned_feature.pth \
    --distillation_type feature \
    --epochs 200 \
    --lr 1e-4 \
    --alpha 0.5 \
    --beta 1

print("\n✅ Feature KD [2/5] 완료!")

In [ ]:
# Feature KD - Round 3/5 (에폭 401-600)
print("🚀 Feature KD [3/5] 시작... (에폭 401-600)")
print("="*60)

!{python_exec} finetune_pruned_model.py \
    --config config_catanet.yml \
    --teacher_weights weights/CATANet-L_x2.pth \
    --pruned_weights weights/catanet_pruned.pth \
    --save_path weights/finetuned_feature.pth \
    --distillation_type feature \
    --epochs 200 \
    --lr 1e-4 \
    --alpha 0.5 \
    --beta 1

print("\n✅ Feature KD [3/5] 완료!")

In [ ]:
# Feature KD - Round 4/5 (에폭 601-800)
print("🚀 Feature KD [4/5] 시작... (에폭 601-800)")
print("="*60)

!{python_exec} finetune_pruned_model.py \
    --config config_catanet.yml \
    --teacher_weights weights/CATANet-L_x2.pth \
    --pruned_weights weights/catanet_pruned.pth \
    --save_path weights/finetuned_feature.pth \
    --distillation_type feature \
    --epochs 200 \
    --lr 1e-4 \
    --alpha 0.5 \
    --beta 1

print("\n✅ Feature KD [4/5] 완료!")

In [ ]:
# Feature KD - Round 5/5 (에폭 801-1000)
print("🚀 Feature KD [5/5] 시작... (에폭 801-1000)")
print("="*60)

!{python_exec} finetune_pruned_model.py \
    --config config_catanet.yml \
    --teacher_weights weights/CATANet-L_x2.pth \
    --pruned_weights weights/catanet_pruned.pth \
    --save_path weights/finetuned_feature.pth \
    --distillation_type feature \
    --epochs 200 \
    --lr 1e-4 \
    --alpha 0.5 \
    --beta 1

print("\n✅ Feature KD [5/5] 완료! (총 1000 에폭)")

In [ ]:
# Feature KD 그래프 표시
import pandas as pd
import matplotlib.pyplot as plt
import os

print("📊 Feature KD 학습 결과")
print("="*60)

log_path = 'results/logs/feature_kd.csv'
if os.path.exists(log_path):
    df = pd.read_csv(log_path)
    print(f"현재 에폭: {len(df)}/1000")
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Loss 그래프
    axes[0].plot(df['epoch'], df['total_loss'], label='Total Loss', color='blue')
    axes[0].plot(df['epoch'], df['task_loss'], label='Task Loss', color='green', alpha=0.7)
    axes[0].plot(df['epoch'], df['distill_loss'], label='Distill Loss', color='red', alpha=0.7)
    axes[0].plot(df['epoch'], df['intermediate_loss'], label='Feature Loss', color='orange', alpha=0.7)
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Feature KD - Training Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # PSNR 그래프
    axes[1].plot(df['epoch'], df['val_psnr'], label='Val PSNR', color='purple', linewidth=2)
    axes[1].axhline(y=df['val_psnr'].max(), color='red', linestyle='--', alpha=0.5, 
                    label=f'Best: {df["val_psnr"].max():.2f}')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('PSNR (dB)')
    axes[1].set_title('Feature KD - Validation PSNR')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📈 Best PSNR: {df['val_psnr'].max():.4f} (에폭 {df.loc[df['val_psnr'].idxmax(), 'epoch']})")
else:
    print("❌ Feature KD 로그 파일이 없습니다.")

---
## 2.5 지식 증류 - FaKD (200 에폭 × 5회 = 1000 에폭)

In [ ]:
# FaKD - Round 1/5 (에폭 1-200)
print("🚀 FaKD [1/5] 시작... (에폭 1-200)")
print("="*60)

!{python_exec} finetune_pruned_model.py \
    --config config_catanet.yml \
    --teacher_weights weights/CATANet-L_x2.pth \
    --pruned_weights weights/catanet_pruned.pth \
    --save_path weights/finetuned_fakd.pth \
    --distillation_type fakd \
    --epochs 200 \
    --lr 1e-4 \
    --alpha 0.5 \
    --beta 100

print("\n✅ FaKD [1/5] 완료!")

In [ ]:
# FaKD - Round 2/5 (에폭 201-400)
print("🚀 FaKD [2/5] 시작... (에폭 201-400)")
print("="*60)

!{python_exec} finetune_pruned_model.py \
    --config config_catanet.yml \
    --teacher_weights weights/CATANet-L_x2.pth \
    --pruned_weights weights/catanet_pruned.pth \
    --save_path weights/finetuned_fakd.pth \
    --distillation_type fakd \
    --epochs 200 \
    --lr 1e-4 \
    --alpha 0.5 \
    --beta 100

print("\n✅ FaKD [2/5] 완료!")

In [ ]:
# FaKD - Round 3/5 (에폭 401-600)
print("🚀 FaKD [3/5] 시작... (에폭 401-600)")
print("="*60)

!{python_exec} finetune_pruned_model.py \
    --config config_catanet.yml \
    --teacher_weights weights/CATANet-L_x2.pth \
    --pruned_weights weights/catanet_pruned.pth \
    --save_path weights/finetuned_fakd.pth \
    --distillation_type fakd \
    --epochs 200 \
    --lr 1e-4 \
    --alpha 0.5 \
    --beta 100

print("\n✅ FaKD [3/5] 완료!")

In [ ]:
# FaKD - Round 4/5 (에폭 601-800)
print("🚀 FaKD [4/5] 시작... (에폭 601-800)")
print("="*60)

!{python_exec} finetune_pruned_model.py \
    --config config_catanet.yml \
    --teacher_weights weights/CATANet-L_x2.pth \
    --pruned_weights weights/catanet_pruned.pth \
    --save_path weights/finetuned_fakd.pth \
    --distillation_type fakd \
    --epochs 200 \
    --lr 1e-4 \
    --alpha 0.5 \
    --beta 100

print("\n✅ FaKD [4/5] 완료!")

In [ ]:
# FaKD - Round 5/5 (에폭 801-1000)
print("🚀 FaKD [5/5] 시작... (에폭 801-1000)")
print("="*60)

!{python_exec} finetune_pruned_model.py \
    --config config_catanet.yml \
    --teacher_weights weights/CATANet-L_x2.pth \
    --pruned_weights weights/catanet_pruned.pth \
    --save_path weights/finetuned_fakd.pth \
    --distillation_type fakd \
    --epochs 200 \
    --lr 1e-4 \
    --alpha 0.5 \
    --beta 100

print("\n✅ FaKD [5/5] 완료! (총 1000 에폭)")

In [ ]:
# FaKD 그래프 표시
import pandas as pd
import matplotlib.pyplot as plt
import os

print("📊 FaKD 학습 결과")
print("="*60)

log_path = 'results/logs/fakd_kd.csv'
if os.path.exists(log_path):
    df = pd.read_csv(log_path)
    print(f"현재 에폭: {len(df)}/1000")
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Loss 그래프
    axes[0].plot(df['epoch'], df['total_loss'], label='Total Loss', color='blue')
    axes[0].plot(df['epoch'], df['task_loss'], label='Task Loss', color='green', alpha=0.7)
    axes[0].plot(df['epoch'], df['distill_loss'], label='Distill Loss', color='red', alpha=0.7)
    axes[0].plot(df['epoch'], df['intermediate_loss'], label='FaKD Loss', color='orange', alpha=0.7)
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('FaKD - Training Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # PSNR 그래프
    axes[1].plot(df['epoch'], df['val_psnr'], label='Val PSNR', color='purple', linewidth=2)
    axes[1].axhline(y=df['val_psnr'].max(), color='red', linestyle='--', alpha=0.5, 
                    label=f'Best: {df["val_psnr"].max():.2f}')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('PSNR (dB)')
    axes[1].set_title('FaKD - Validation PSNR')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📈 Best PSNR: {df['val_psnr'].max():.4f} (에폭 {df.loc[df['val_psnr'].idxmax(), 'epoch']})")
else:
    print("❌ FaKD 로그 파일이 없습니다.")

---
## 2.6 모델 재구성 (Rebuild)

In [ ]:
print("🚀 [STEP 3] 모델 재구성 시작...")
print("="*60)

rebuild_configs = [
    ('finetuned_output.pth', 'rebuilt_output.pth'),
    ('finetuned_feature.pth', 'rebuilt_feature.pth'),
    ('finetuned_fakd.pth', 'rebuilt_fakd.pth'),
]

for source, target in rebuild_configs:
    source_path = f'weights/{source}'
    target_path = f'weights/{target}'
    
    if os.path.exists(source_path):
        print(f"\n🔧 {source} -> {target}")
        !{python_exec} analysis/rebuild_pruned_model.py \
            --config config_catanet.yml \
            --masks weights/catanet_pruning_masks.pth \
            --source_weights {source_path} \
            --save_path {target_path}
    else:
        print(f"⚠️ {source} 없음, 건너뜀")

print("\n✅ 재구성 완료")

---
## 2.7 최종 비교 그래프 (1000 에폭 완료 시)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

print("📊 3가지 KD 방법 비교")
print("="*60)

kd_types = ['output', 'feature', 'fakd']
colors = {'output': 'blue', 'feature': 'green', 'fakd': 'red'}
labels = {'output': 'Output KD', 'feature': 'Feature KD', 'fakd': 'FaKD'}

all_logs = {}
all_complete = True

# 로그 파일 로드
for kd_type in kd_types:
    log_path = f'results/logs/{kd_type}_kd.csv'
    if os.path.exists(log_path):
        df = pd.read_csv(log_path)
        all_logs[kd_type] = df
        status = "✅" if len(df) >= 1000 else "⏳"
        print(f"  {status} {labels[kd_type]}: {len(df)}/1000 에폭")
        if len(df) < 1000:
            all_complete = False
    else:
        print(f"  ❌ {labels[kd_type]}: 로그 없음")
        all_complete = False

if len(all_logs) > 0:
    # PSNR 비교 그래프
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    for kd_type, df in all_logs.items():
        axes[0].plot(df['epoch'], df['val_psnr'], label=labels[kd_type], 
                     color=colors[kd_type], linewidth=1.5, alpha=0.8)
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('PSNR (dB)', fontsize=12)
    axes[0].set_title('KD Methods Comparison - PSNR', fontsize=14)
    axes[0].legend(fontsize=11)
    axes[0].grid(True, alpha=0.3)
    
    # Loss 비교 그래프
    for kd_type, df in all_logs.items():
        axes[1].plot(df['epoch'], df['total_loss'], label=labels[kd_type], 
                     color=colors[kd_type], linewidth=1.5, alpha=0.8)
    axes[1].set_xlabel('Epoch', fontsize=12)
    axes[1].set_ylabel('Total Loss', fontsize=12)
    axes[1].set_title('KD Methods Comparison - Loss', fontsize=14)
    axes[1].legend(fontsize=11)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 최종 요약
    print("\n" + "="*60)
    print("📈 최종 성능 요약")
    print("="*60)
    for kd_type, df in all_logs.items():
        best_psnr = df['val_psnr'].max()
        best_epoch = df.loc[df['val_psnr'].idxmax(), 'epoch']
        print(f"  {labels[kd_type]:12} | Best PSNR: {best_psnr:.4f} (에폭 {best_epoch})")

if all_complete:
    print("\n🎉 모든 KD 방법 1000 에폭 학습 완료!")
else:
    print("\n⏳ 아직 학습이 완료되지 않은 KD 방법이 있습니다.")

---
## 2.8 학습 결과 요약

In [ ]:
import pandas as pd
from IPython.display import display
import os

print("="*60)
print("📊 학습 결과 요약")
print("="*60)

# 로그 파일 확인
print("\n📁 학습 로그 (results/logs/):")
for kd_type in ['output', 'feature', 'fakd']:
    log_path = f'results/logs/{kd_type}_kd.csv'
    if os.path.exists(log_path):
        df = pd.read_csv(log_path)
        print(f"  ✅ {kd_type}_kd.csv: {len(df)}/1000 에폭")
    else:
        print(f"  ❌ {kd_type}_kd.csv: 없음")

# 가중치 파일 확인
print("\n📁 생성된 가중치 파일:")
!ls -lh weights/*.pth 2>/dev/null

# 요약 파일 확인
summary_path = 'results/summary/training_summary.csv'
if os.path.exists(summary_path):
    print("\n📊 최종 학습 요약:")
    summary_df = pd.read_csv(summary_path)
    display(summary_df)
else:
    print("\n⏳ 최종 요약: 3가지 KD 방법 모두 1000 에폭 완료 시 생성됩니다.")

print("\n" + "="*60)
print("✅ 학습 파이프라인 완료!")
print("💡 다음 단계: 3_benchmark.ipynb에서 성능 평가")
print("="*60)